In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import StandardScaler

# -----------------------------------------------------
# STEP 1: Load all CSV files
# -----------------------------------------------------
batting = pd.read_csv("Batting_Data.csv")
bowling = pd.read_csv("Bowling_Data.csv")
fielding = pd.read_csv("Fielding_Data.csv")
allround = pd.read_csv("AllRounder_Data.csv")

# -----------------------------------------------------
# STEP 2: Clean Player Names and Extract Country
# -----------------------------------------------------
def clean_player_name(name):
    if pd.isna(name):
        return None
    return re.sub(r"\s*\([^)]*\)", "", str(name)).strip()

def extract_country(name):
    if pd.isna(name):
        return None
    match = re.search(r"\((.*?)\)", str(name))
    return match.group(1) if match else None

for df in [batting, bowling, fielding, allround]:
    df["Player_Name"] = df["Player"].apply(clean_player_name)
    df["Country"] = df["Player"].apply(extract_country)
    df.drop(columns=["Player"], inplace=True, errors="ignore")

# Remove '/ICC' from Country
for df in [batting, bowling, fielding, allround]:
    if 'Country' in df.columns:
        df['Country'] = df['Country'].str.replace('ICC/', '', regex=False)\
                                     .str.replace('/ICC', '', regex=False).str.strip()

# -----------------------------------------------------
# STEP 3: Merge datasets by Player_Name and Player_ID
# -----------------------------------------------------
merged = pd.merge(batting, bowling, on=["Player_Name", "Player_ID"], how="outer", suffixes=("_bat", "_bowl"))
merged = pd.merge(merged, fielding, on=["Player_Name", "Player_ID"], how="outer", suffixes=("", "_field"))

# Career_Span, Matches_Played, Country from available datasets
merged["Career_Span"] = (
    merged.get("Career_Span_bat", pd.Series())
    .combine_first(merged.get("Career_Span_bowl", pd.Series()))
    .combine_first(merged.get("Career_Span", pd.Series()))
)

merged["Matches_Played"] = (
    merged.get("Matches_Played_bat", pd.Series())
    .combine_first(merged.get("Matches_Played_bowl", pd.Series()))
    .combine_first(merged.get("Matches_Played", pd.Series()))
)

merged["Country"] = (
    merged.get("Country_bat", pd.Series())
    .combine_first(merged.get("Country_bowl", pd.Series()))
    .combine_first(merged.get("Country", pd.Series()))
)

# Batting & Fielding stats
merged["Innings_Batted"] = merged.get("Innings_Batted_bat", merged.get("Innings_Batted", 0))
merged["Total_Runs"] = merged.get("Total_Runs_bat", merged.get("Total_Runs", 0))
merged["Stumpings_Total"] = merged.get("Stumpings_Total", 0)
merged["Catches_Total"] = merged.get("Catches_Total_field", merged.get("Catches_Total", 0))

# -----------------------------------------------------
# STEP 4: Calculate Fielding_Catches
# -----------------------------------------------------
merged["Fielding_Catches"] = merged["Catches_Total"] - merged["Stumpings_Total"]

# -----------------------------------------------------
# STEP 5: Active Player
# -----------------------------------------------------
def is_active(span):
    if pd.isna(span):
        return "No"
    return "Yes" if str(span).endswith("2024") or str(span).endswith("2025") else "No"

merged["Active_Player"] = merged["Career_Span"].apply(is_active)

# -----------------------------------------------------
# STEP 6: Is_Allrounder
# -----------------------------------------------------
allrounder_set = set(allround["Player_Name"].dropna())
merged["Is_Allrounder"] = merged["Player_Name"].apply(lambda x: "Yes" if x in allrounder_set else "No")

# -----------------------------------------------------
# STEP 7: Normalize Batting and Bowling Averages for Player_Category
# -----------------------------------------------------
for col in ["Batting_Average", "Bowling_Average"]:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors="coerce")

bat_avg = merged["Batting_Average"].fillna(merged["Batting_Average"].mean())
bowl_avg = merged["Bowling_Average"].fillna(merged["Bowling_Average"].mean())

merged["bat_z"] = StandardScaler().fit_transform(bat_avg.values.reshape(-1,1))
merged["bowl_z"] = -StandardScaler().fit_transform(bowl_avg.values.reshape(-1,1))  # invert logic

def assign_category(row):
    stumpings = row.get("Stumpings_Total", 0)
    bat_z = row.get("bat_z", 0)
    bowl_z = row.get("bowl_z", 0)
    
    if pd.notna(stumpings) and stumpings > 0:
        return "Wicketkeeper"
    elif pd.notna(bat_z) and pd.notna(bowl_z):
        return "Batsman" if bat_z > bowl_z else "Bowler"
    elif pd.notna(bat_z):
        return "Batsman"
    elif pd.notna(bowl_z):
        return "Bowler"
    else:
        return "Unknown"

merged["Player_Category"] = merged.apply(assign_category, axis=1)

# -----------------------------------------------------
# STEP 8: Drop duplicates, reset index, assign ID
# -----------------------------------------------------
merged = merged.drop_duplicates(subset=["Player_Name"], keep="first").reset_index(drop=True)
merged["ID"] = merged.index + 1

# -----------------------------------------------------
# STEP 9: Fill missing numeric columns with median or 0
# -----------------------------------------------------
numeric_cols = [
    'Stumpings_Total', 'Catches_Total', 'Fielding_Catches', 'Runs_Conceded', 'Wickets_Total',
    'Batting_Average', 'Bowling_Average', 'Economy_Rate', 'Strike_Rate', 'Five_Wickets',
    'Ten_Wickets', 'Innings_Fielded', 'Innings_Batted', 'Total_Runs'
]

for col in numeric_cols:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors="coerce").fillna(0)

# -----------------------------------------------------
# STEP 10: Keep only required columns
# -----------------------------------------------------
final_cols = [
    "ID", "Player_ID", "Player_Name", "Country", "Career_Span", "Matches_Played",
    "Active_Player", "Is_Allrounder", "Player_Category", "Innings_Batted", "Total_Runs", 
    "Batting_Average", "Not_Outs", "Highest_Score", "Centuries", "Fifties", "Ducks_Career",
    "Stumpings_Total", "Catches_Total", "Fielding_Catches", "Runs_Conceded", "Wickets_Total",
    "Bowling_Average", "Economy_Rate", "Strike_Rate", "Five_Wickets", "Ten_Wickets", "Innings_Fielded"
]

final = merged[[c for c in final_cols if c in merged.columns]]

# -----------------------------------------------------
# STEP 11: Save final dataset
# -----------------------------------------------------
final.to_csv("Final_Player_Stats.csv", index=False)
print(f"✅ Final dataset created: {final.shape[0]} players saved to Final_Player_Stats.csv")
print(final["Player_Category"].value_counts())
print(final.head(10))


✅ Final dataset created: 3228 players saved to Final_Player_Stats.csv
Player_Category
Bowler          1747
Batsman         1251
Wicketkeeper     230
Name: count, dtype: int64
   ID  Player_ID  Player_Name Country Career_Span  Matches_Played  \
0   1    1078692   A Athanaze      WI   2023-2025              15   
1   2      44026     A Bacher      SA   1965-1970              12   
2   3     303423  A Balbirnie     IRE   2018-2025              11   
3   4       9249      A Brown     ENG   1961-1961               2   
4   5      27639     A Chopra     IND   2003-2004              10   
5   6       4611  A Coningham     AUS   1894-1895               1   
6   7       4632     A Cotter     AUS   1904-1912              21   
7   8      10850      A Coxon     ENG   1948-1948               1   
8   9      40041    A D'Souza     PAK   1959-1962               6   
9  10     574178  A Dananjaya      SL   2018-2019               6   

  Active_Player Is_Allrounder Player_Category  Innings_Batted  ..